In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import os
import glob
import re

In [30]:
# Concating raw data
path = r"D:\Python\Python project\Product_Tree"
all_dfs = []
for file in glob.glob(os.path.join(path, "Product_Tree_*.xlsb")):
    # Extract year from filename using regex
    filename = os.path.basename(file)
    match = re.search(r"(\d{4})", filename)
    year = match.group(1) if match else None
    # Read the "DATA" sheet
    df_temp = pd.read_excel(file, sheet_name="DATA", engine="pyxlsb")
    # Add year column
    df_temp["applied_year"] = year
    all_dfs.append(df_temp)

# Concatenate all DataFrames
df = pd.concat(all_dfs, ignore_index=True)

In [31]:
# Drop unnamed columns & unneccessary columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns=['KIỂU GIÀY CŨ', 'LOẠI ĐẾ ', 'THỜI GIAN BÁN CHTT',
       'Nhóm tương đồng cũ', 'Mã tính KHM', 'Mã hóa chất liệu  khuôn đế',
       'Mã hóa chất liệu mũ quai', 'Độc quyền B2B'])

In [32]:
# Realign Column names:
df = df.rename(columns={"Style color":"style_color", 
                        "Color Group":"color_group",
                        "Listing Price":"listing_price",
                        "Price Group":"price_group",
                        "Product Group":"product_group",
                        "Detailed Product Group":"detailed_product_group",
                        "Shoe Product":"shoe_product",
                        "Specialized Function":"specialized_function",
                        "Dedicated functions":"dedicated_functions",
                        "Size Group":"size_group",
                        "Size Range":"size_range",
                        "Age Group":"age_group",
                        "Activity Group":"activity_group",
                        "Copyright group":"copyright_group",
                        "Image copyright":"image_copyright",
                        "Product Line":"product_line",
                        "Design Inspiration":"design_inspiration",
                        "Lifestyle Group":"lifestyle_group",
                        "Launch Season":"launch_season",
                        "Similar group":"similar_group",
                        "Mold code":"mold_code",
                        "Heel Height":"heel_height",
                        "Upper_material":"upper_material",
                        "Design team":"design_team",
                        "OPTION NAME":"option_name",
                        "Cost price":"cost_price",
                        "Brand":"brand",
                        "Gender":"gender",
                        "Style":"style",
                        "Color":"color",
                        "Size":"size",
                        "Vendor":"vendor",
                        "OPTION":"option"})

In [33]:
df.columns

Index(['SKU', 'style_color', 'style', 'color', 'color_group', 'listing_price',
       'price_group', 'brand', 'gender', 'product_group',
       'detailed_product_group', 'shoe_product', 'specialized_function',
       'dedicated_functions', 'size_group', 'size', 'size_range', 'age_group',
       'activity_group', 'Collection', 'copyright_group', 'image_copyright',
       'product_line', 'design_inspiration', 'lifestyle_group',
       'launch_season', 'similar_group', 'mold_code', 'heel_height',
       'upper_material', 'design_team', 'vendor', 'option', 'option_name',
       'cost_price', 'applied_year'],
      dtype='object')

In [34]:
for columns in df.columns:
    print(f"Null value quantity in {columns}: {df[columns].isnull().sum()}")

Null value quantity in SKU: 0
Null value quantity in style_color: 0
Null value quantity in style: 0
Null value quantity in color: 155
Null value quantity in color_group: 0
Null value quantity in listing_price: 242
Null value quantity in price_group: 370
Null value quantity in brand: 0
Null value quantity in gender: 0
Null value quantity in product_group: 0
Null value quantity in detailed_product_group: 0
Null value quantity in shoe_product: 0
Null value quantity in specialized_function: 0
Null value quantity in dedicated_functions: 85
Null value quantity in size_group: 0
Null value quantity in size: 326
Null value quantity in size_range: 7422
Null value quantity in age_group: 6
Null value quantity in activity_group: 0
Null value quantity in Collection: 3233
Null value quantity in copyright_group: 3
Null value quantity in image_copyright: 2168
Null value quantity in product_line: 0
Null value quantity in design_inspiration: 21865
Null value quantity in lifestyle_group: 716
Null value qu

In [37]:
# Define all mappings in a single dictionary
mappings = {
    "color_group": {
        "TỐI": "dark", "SÁNG": "light", "NHIỀU MÀU": "colorful",
        "ĐEN": "black", "KHÁC": "other", "TRẮNG": "white", "MÀU SẮC": "colorful"
    },
    
    "gender": {
        "WOM":"women","MEN":"men","GIR":"gir","BOY":"boy"
    },
    
    "product_group": {
        "DEP":"slipper","SAN":"sandal","GIA":"shoe","PKI":"accessories", "TBL":"backpack"
    },
    
    "detailed_product_group": {
        "DEPTD":"popular_slipper","SANTD":"popular_sandal","GTTPC":"popular_sport_shoe","GIATR":"fashion_shoes","SANTR":"fashion_sandal",
        "GTTCD":"specialized_sport_shoes","PKIEN":"accessories","TBLTR":"fashion_backpack","TBLTH":"hunter_backpack"
    },
    
    "shoe_product": {
        "DXP":"foam_slipper","STD":"popular_sandal","GSK":"sneaker_shoe","STT":"sports_sandal","DLA":"lao_slipper",
        "SXP":"foam_sandal","STR":"fashion_sandal","GBB":"doll_shoe","GBI":"closed_shoe","DTD":"popular_slipper",
        "GTY":"western_shoe","DVP":"evaphun_slipper","GTD":"popular_shoe","TBLTR":"fashion_backpack","DTT":"sports_slipper",
        "GTS":"tennis_shoe","PKV":"sock_accessories","GFB":"football_shoe","DHA":"slippers","PKIEN":"accessories","PKM":"face_accessories",
        "GCH":"running_shoe","PKG":"shoe_accessories","GDB":"walking_shoe","GBT":"boots_shoe","PKP":"office_accessories","TBLTH":"hunter_backpack",
        "PKK":"other_accessories","GYT":"medical_shoe","DNP":"spray_plastic_slipper","GBH":"safety_shoe"},
    
    "specialized_function": {
        "Hàng ngày": "daily_use", "Bóng đá": "football", "Đi bộ": "walking",
        "Chạy bộ": "jogging", "Y tế": "medical", "Công trình": "construction",
        "Khiêu vũ": "dancing","Tennis":"tennis"
    },
    
    "dedicated_functions": {
        "Không có": "none", "Sân đất nện": "clay_court", "Sân cỏ nhân tạo": "artificial_turf",
        "Giày đi bộ": "walking_shoes", "20km": "20km", "Sân futsal": "futsal_court",
        "40km": "40km", "Sân Futsal": "futsal_court", "Giày bảo hộ": "safety_shoes",
        "10km": "10km"
    },
    
    "size_group": {
        "Ngoại lệ": "exception", "Khung size chính": "main_size_frame",
        "Khác": "other", "Đặc biệt": "special"
    },
    
    "activity_group": {
        "Thường nhật/Trường học": "Daily_School", "Văn phòng": "office",
        "Khác": "other", "Thể thao": "sport", "Chuyên biệt": "specialized"
    },
    
    "copyright_group": {
        "CƠ BẢN": "basic","DORAEMON":"doraemon","DISNEY":"disney","DC":"dc","MARVEL":"marvel","TIDU":"tidu"
    },
    
    "image_copyright": {
        "CƠ BẢN": "basic", "KHÁC": "other", "KHÔNG": "none", "KHÔNG CÓ": "none",
        "VƯỜN RAU CỦA MẸ": "mother_vegetable_garden",
        "GẤU CHÚ TÂM": "bear_mind", "Gấu Chú Tâm": "bear_mind"
    },
    "product_line": {
        "KHÁC": "OTHER", "ÊMBRACE": "EMBRACE"
    }
}

# Apply mappings
for col, mapping in mappings.items():
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace(mapping)

# Special handling for Age Group (regex-based replacements)
replacements = {
    "đến": "To",
    "tuổi": "yrs_old",
    "Trên": "Above",
    "Khác": "Other"
}
if "age_group" in df.columns:
    df["age_group"] = df["age_group"].astype(str).str.strip().replace(replacements, regex=True)

In [29]:
df["product_line"].value_counts()

product_line
KHÁC              364854
X                   8813
CORE                6888
STREET              4600
PERFORMANCE         3341
COOL KIDS CLUB      2290
ÊMBRACE             1017
HAPPY SCHOOL         602
HELLIO                66
BALO                   6
Name: count, dtype: int64

In [38]:
df

,SKU,style_color,style,color,color_group,listing_price,price_group,brand,gender,product_group,...,similar_group,mold_code,heel_height,upper_material,design_team,vendor,option,option_name,cost_price,applied_year
0,DRM025010DEN38,DRM025010DEN,DRM025010,DEN,dark,260000.0,200<300,BITI'S,men,sandal,...,QLNQ,BT057,Đế Sẹp,QSI,NaN,BITI'S,P03-C05,200<300-TỐI,176800.0,2022
1,DRM025010DEN39,DRM025010DEN,DRM025010,DEN,dark,260000.0,200<300,BITI'S,men,sandal,...,QLNQ,BT057,Đế Sẹp,QSI,NaN,BITI'S,P03-C05,200<300-TỐI,176800.0,2022
2,DRM025010DEN40,DRM025010DEN,DRM025010,DEN,dark,260000.0,200<300,BITI'S,men,sandal,...,QLNQ,BT057,Đế Sẹp,QSI,NaN,BITI'S,P03-C05,200<300-TỐI,176800.0,2022
3,DRM025010DEN41,DRM025010DEN,DRM025010,DEN,dark,260000.0,200<300,BITI'S,men,sandal,...,QLNQ,BT057,Đế Sẹp,QSI,NaN,BITI'S,P03-C05,200<300-TỐI,176800.0,2022
4,DRM025010DEN42,DRM025010DEN,DRM025010,DEN,dark,260000.0,200<300,BITI'S,men,sandal,...,QLNQ,BT057,Đế Sẹp,QSI,NaN,BITI'S,P03-C05,200<300-TỐI,176800.0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392472,AAUH05200XNG,AAUH05200XNG,AAUH05200,XNG,colorful,NaN,NaN,BITI'S,boy,accessories,...,CHAM,NaN,NaN,NaN,Outsource,NaN,NaN,NaN,NaN,2025
392473,ASGG00120DEN,ASGG00120DEN,ASGG00120,DEN,black,NaN,NaN,BITI'S,boy,accessories,...,VCCV,NaN,NaN,NaN,Outsource,NaN,NaN,NaN,NaN,2025
392474,ASGG00120TRG,ASGG00120TRG,ASGG00120,TRG,white,NaN,NaN,BITI'S,boy,accessories,...,VCCV,NaN,NaN,NaN,Outsource,NaN,NaN,NaN,NaN,2025
392475,AAUH05900VAG,AAUH05900VAG,AAUH05900,VAG,light,NaN,NaN,BITI'S,boy,accessories,...,PKTB,NaN,NaN,NaN,Outsource,NaN,NaN,NaN,NaN,2025
